# Setting up the Environment

Ensure by running nvidia-smi and nvcc --version that that the system has CUDA version 11.2+ (especially for the CUDA compiler) and NVIDIA driver version 460+. Use Colab's 'Runtime -> Change runtime type' menu to select a GPU environment. You will have to choose high RAM for data set generation, but that comes at the cost of disabling of background execution.

In [ ]:
!nvidia-smi

Mon Mar  7 17:11:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# should not be less than CUDA 11.1
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


Install some essential utilities listed on the Instant-NGP github page. Also install latest version of cmake using pip (apt has an outdated version) - should be cmake 3.21+.

In [ ]:
!apt update && apt install build-essential git python3-dev python3-pip libopenexr-dev libxi-dev libglfw3-dev libglew-dev libomp-dev libxinerama-dev libxcursor-dev
!pip install --upgrade cmake

# Setting up the Instant-NGP Codebase

Clone the Instant NGP github repository. Now change to the repo folder, which should be /content/instant-ngp in a colab environment.

In [ ]:
# clone the repository
!git clone --recursive https://github.com/nvlabs/instant-ngp
%cd instant-ngp

In [ ]:
# ensure that we are inside the instant-ngp directory
# befor proceeding to compilation
!pwd

/content/instant-ngp


Important step: model compilation
Build the model with GUI off. GUI version is currently not supported for Jupyter based notebooks and headless sessions. You may have to change the Tiny CUDA environment (%env TCNN_CUDA_ARCHITECTURE=<your-2-digit-GPU-series>). Refer to the Instant-NGP github page for more details.

Please refer to the Training and Rendering notebook for sample correct outputs for these steps.

In [ ]:
#%env TCNN_CUDA_ARCHITECTURES=75

# model compilation setup without GUI
!cmake . -B build -DNGP_BUILD_WITH_GUI=OFF

Be patient with this step. May take more than 15 minutes on low RAM environments. Compilation time can be reduced to 3-5 minutes on high RAM GPUs.

Ignore the warnings of the compiler, but ensure that the build is 100% completed.

In [ ]:
# actual model compilation
# the current bottleneck in terms of time
!cmake --build build --config RelWithDebInfo -j `nproc`

In [ ]:
# install the python library dependencies
!pip3 install -r requirements.txt

Install the python libraries at versions compatible with the Instant-NGP code, consulting the requirements.txt file. Restart the system after this step for successful installation.

In [ ]:
# ensure that we are within the instant-ngp folder
# before testing the model
%cd /content/instant-ngp
!pwd

/content/instant-ngp
/content/instant-ngp


To test if the model is working (and hence ready for training and rendering), first change to /content/instant-ngp. Then train the model on author's Fox dataset. The training process will not terminate and has to be stopped manually. You should be able to see iterations and corresponding losses in training if your model is functioning correctly.

In [ ]:
!./build/testbed --scene data/nerf/fox

17:19:43 INFO     Loading NeRF dataset from
17:19:43 INFO       data/nerf/fox/transforms.json
17:19:43 PROGRESS []   0% ( 0/50)  0s/inf17:19:43 PROGRESS []   2% ( 1/50) 0s/1s17:19:43 PROGRESS []   4% ( 2/50) 0s/0s17:19:43 PROGRESS []   6% ( 3/50) 0s/0s17:19:43 PROGRESS []   8% ( 4/50) 0s/0s17:19:43 PROGRESS []  10% ( 5/50) 0s/0s17:19:43 PROGRESS []  12% ( 6/50) 0s/0s17:19:43 PROGRESS []  14% ( 7/50) 0s/0s17:19:43 PROGRESS []  16% ( 8/50) 0s/0s17:19:43 PROGRESS []  18% ( 9/50) 0s/0s17:19:43 PROGRESS []  20% (10/50) 0s/0s17:19:43 PROGRESS []  22% (11/50) 0s/0s17:19:43 PROGRESS []  24% (12/50) 0s/0s17:19:43 PROGRESS []  26% (13/50) 0s/0s17:19:43 PROGRESS []  28% (14/50) 0s/0s17:19:43 PROGRESS []  30% (15/50) 0s/0s17:19:43 PROGRESS []  32% (16/50) 0s/0s17:19:43 PROGRESS []  34% (17/50) 0s/0s17:19:43 PROGRESS []  36% (18/50) 0s/0s17:19:43 PROGRESS []  38% (19/50) 0s/0s17:19:43 PROGRESS []  40% (20/50) 0s/0s17:19:43 PROGRESS []  42% (21/50) 0s/0s17:19:43 PROGRESS []  44% (22/50) 0s/0s17:19:4

# COLMAP Setup

Clone the Colmap repository from github to your /content folder.

In [ ]:
%cd /content
!pwd

/content
/content


In [ ]:
!git clone https://github.com/colmap/colmap

Cloning into 'colmap'...
remote: Enumerating objects: 14012, done.
remote: Counting objects: 100% (983/983), done.
remote: Compressing objects: 100% (638/638), done.
remote: Total 14012 (delta 567), reused 520 (delta 344), pack-reused 13029
Receiving objects: 100% (14012/14012), 12.40 MiB | 25.39 MiB/s, done.
Resolving deltas: 100% (10842/10842), done.


Install some essential dependencies for Colmap using apt. Refer to https://colmap.github.io/install.html for further details.

In [ ]:
!sudo apt-get install libboost-program-options-dev libboost-filesystem-dev libboost-graph-dev libboost-system-dev libboost-test-dev libeigen3-dev libsuitesparse-dev libfreeimage-dev libgoogle-glog-dev libgflags-dev libglew-dev qtbase5-dev libqt5opengl5-dev libcgal-dev

In [ ]:
!sudo apt-get install libcgal-qt5-dev

In [ ]:
!sudo apt-get install libatlas-base-dev libsuitesparse-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsuitesparse-dev is already the newest version (1:5.1.2-2).
libatlas-base-dev is already the newest version (3.10.3-5).
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 78 not upgraded.


**Ceres Solver**: Colmap requires Ceres Solver for model build. Clone ceres-solver from github to your /content, and ensure your cloned repo is up to date.

In [ ]:
!git clone https://ceres-solver.googlesource.com/ceres-solver

Cloning into 'ceres-solver'...
remote: Sending approximately 18.10 MiB ...
remote: Counting objects: 33, done
remote: Total 17641 (delta 10640), reused 17641 (delta 10640)
Receiving objects: 100% (17641/17641), 18.10 MiB | 31.53 MiB/s, done.
Resolving deltas: 100% (10640/10640), done.


In [ ]:
%cd /content/ceres-solver
!pwd

/content/ceres-solver
/content/ceres-solver


In [ ]:
!git checkout $(git describe --tags)

Note: checking out '2.1.0rc1-1-g94af0918'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 94af0918 Force C++ linker


In /content/ceres-solver, create an empty directory "build", and cmake to compile ceres-solver code into it. Then install ceres-solver using the compiled build.

In [ ]:
!mkdir build
%cd build
!pwd

/content/ceres-solver/build
/content/ceres-solver/build


In [ ]:
!cmake .. -DBUILD_TESTING=OFF -DBUILD_EXAMPLES=OFF

In [ ]:
!make -j

In [ ]:
!sudo make install

Clone NVIDIA's libglvnd from github to your /content. Install it using "make" and "make install".

In [ ]:
%cd /content
!pwd

/content
/content


In [ ]:
!git clone https://github.com/NVIDIA/libglvnd
!sudo apt-get install libxext-dev libx11-dev x11proto-gl-dev

In [ ]:
%cd libglvnd/
!pwd

/content/libglvnd
/content/libglvnd


In [ ]:
!sudo apt-get install autoconf automake libtool
!sudo apt-get install libffi-dev
!./autogen.sh
!./configure
!make  -j4
!sudo make install

Finally, check that you are on dev branch of Colmap (master outdated). Install libmetis-dev. Compile and install Colmap with cmake.

In [ ]:
%cd /content
!pwd

/content
/content


In [ ]:
!git clone https://github.com/colmap/colmap.git

fatal: destination path 'colmap' already exists and is not an empty directory.


In [ ]:
%cd /content/colmap
!pwd

/content/colmap
/content/colmap


In [ ]:
!git checkout dev

Already on 'master'
Your branch is up to date with 'origin/master'.


In [ ]:
!mkdir build
%cd build
!pwd

/content/colmap/build
/content/colmap/build


In [ ]:
!sudo apt-get install libmetis-dev

In [ ]:
!cmake ..

In [ ]:
!make -j

In [ ]:
!sudo make install

In [ ]:
!colmap -h

# FFMPEG Setup

In case you are using a video for dataset generation, Colmap will use FFMEPG to convert the video into images (frames). You will need to install ffmepg with apt as below.

In [ ]:
!sudo apt update
!sudo apt install ffmpeg
!ffmpeg -version
!ffmpeg -?

# Dataset Generation

Come to your /content/instant-ngp folder. Mount the google drive to /content. It will be available as /content/drive/MyDrive.

In [ ]:
%cd /content/instant-ngp
!pwd

/content/instant-ngp
/content/instant-ngp


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Use "scripts/colmap2nerf.py" to convert the video to a dataset with images and transforms.json. Specify path to video with "--video_in <path-to-video>", fps with "--video-fps <video-fps>", and bounding box size (1, 2, 4, 8, or 16) with "--aabb_scale <box-size>". Note the sizes 8 and 16 might not be supported during training in colab environment due to their exorbitant memory requirements.

This step should take around 10 minutes. Please take care to download or make note of the location of the transforms.json file which contains all the camera information and tranformation matrices. Also note that you may change the camera parameters like field of view or focal length during data set generation interactively by using the input text box (run cell below).

In [ ]:
!python scripts/colmap2nerf.py --video_in /content/drive/MyDrive/goku.mp4 --video_fps 2 --run_colmap --aabb_scale 16

Streaming output truncated to the last 5000 lines.

  => Image sees 562 / 1138 points

Pose refinement report
----------------------
    Residuals : 1144
   Parameters : 6
   Iterations : 7
         Time : 0.012197 [s]
 Initial cost : 0.678036 [px]
   Final cost : 0.469382 [px]
  Termination : Convergence

  => Continued observations: 557
  => Added observations: 401

Bundle adjustment report
------------------------
    Residuals : 9568
   Parameters : 1064
   Iterations : 26
         Time : 0.318311 [s]
 Initial cost : 0.56539 [px]
   Final cost : 0.554944 [px]
  Termination : No convergence

  => Merged observations: 96
  => Completed observations: 93
  => Filtered observations: 43
  => Changed observations: 0.042251

Bundle adjustment report
------------------------
    Residuals : 9740
   Parameters : 983
   Iterations : 3
         Time : 0.0377491 [s]
 Initial cost : 0.681417 [px]
   Final cost : 0.673879 [px]
  Termination : Convergence

  => Merged observations: 7
  => Complete

You may try out your newly generated dataset as below. Please refer to the Training and Rendering notebook for further details.

In [ ]:
!python3 scripts/run.py --scene="/content/instant-ngp/transforms.json" --mode=nerf --screenshot_transforms="/content/instant-ngp/transforms.json" --screenshot_w=1080 --screenshot_h=1920 --screenshot_dir="data/screenshots" --save_snapshot=data/snapshot.msgpack --n_steps=500